In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# CSV 파일 불러오기
tl_path = "/content/drive/MyDrive/TL_tn_activity_consume_his_활동소비내역_G.csv"
vl_path = "/content/drive/MyDrive/VL_tn_activity_consume_his_활동소비내역_G.csv"

df_tl = pd.read_csv(tl_path)
df_vl = pd.read_csv(vl_path)

# 두 df 합치기
df = pd.concat([df_tl, df_vl], ignore_index=True)

# 필요한 열만 선택
df = df[[
    "BRNO",                   # 사업자등록번호
    "STORE_NM",               # 상호명
    "PAYMENT_NUM",            # 소비인원
    "PAYMENT_DT",             # 결제일시_분
    "PAYMENT_AMT_WON",        # 결제금액_원
    "ACTIVITY_TYPE_CD",       # 활동유형코드
    "ROAD_NM_CD"              # 도로명코드
]]

# df 확인
df

,BRNO,STORE_NM,PAYMENT_NUM,PAYMENT_DT,PAYMENT_AMT_WON,ACTIVITY_TYPE_CD,ROAD_NM_CD
0,1.128131e+09,아르떼뮤지엄여수,4,2023-05-19 12:01,0.0,3,4647160.0
1,5.725101e+09,우리동네와인슈퍼,2,2023-06-08 13:37,24000.0,2,3254184.0
2,2.082693e+09,갤러리카페버그내길135,2,2023-06-08 14:39,12000.0,1,3265126.0
3,8.825201e+09,호도수꽃산장,2,2023-05-20 18:04,14000.0,1,3346009.0
4,3.178134e+09,농업회사법인괴산한우타운,2,2023-06-17 14:53,22000.0,1,3246004.0
...,...,...,...,...,...,...,...
17711,5.111096e+09,반하담,2,2023-09-29 17:51,10000.0,2,4853861.0
17712,2.476700e+09,모란창평국밥,1,2023-09-28 11:38,9000.0,1,4658020.0
17713,4.098215e+09,사단법인 아시아차문화연구회,1,2023-09-28 12:40,4500.0,1,4658020.0
17714,4.098307e+09,한국대나무박물관,1,2023-09-28 10:22,2000.0,3,3286016.0


In [3]:
# 데이터타입 확인
print(f'데이터타입 : {df.dtypes}')

데이터타입 : BRNO                float64
STORE_NM             object
PAYMENT_NUM           int64
PAYMENT_DT           object
PAYMENT_AMT_WON     float64
ACTIVITY_TYPE_CD      int64
ROAD_NM_CD          float64
dtype: object


In [4]:
# PAYMENT_DT를 datetime으로 변환
df["PAYMENT_DT"] = pd.to_datetime(df["PAYMENT_DT"], format="%Y-%m-%d %H:%M")

# 주말 여부 컬럼 추가 (토/일 → 1, 평일 → 0) : IS_WEEKEND
# 월:0, 화:1, 수:2, 목:3, 금:4, 토:5, 일: 6
df["IS_WEEKEND"] = df["PAYMENT_DT"].dt.weekday >= 5
df["IS_WEEKEND"] = df["IS_WEEKEND"].astype(int)

# 시간대 분류 컬럼 추가 : TIME_SLOT
# 새벽(0~6시), 오전(6~12시), 오후(12~18시), 밤(18~0시)
def get_time_slot(hour):
    if hour < 6:
        return "dawn"
    elif hour < 12:
        return "morning"
    elif hour < 18:
        return "afternoon"
    else:
        return "night"

df["TIME_SLOT"] = df["PAYMENT_DT"].dt.hour.apply(get_time_slot)

df

,BRNO,STORE_NM,PAYMENT_NUM,PAYMENT_DT,PAYMENT_AMT_WON,ACTIVITY_TYPE_CD,ROAD_NM_CD,IS_WEEKEND,TIME_SLOT
0,1.128131e+09,아르떼뮤지엄여수,4,2023-05-19 12:01:00,0.0,3,4647160.0,0,afternoon
1,5.725101e+09,우리동네와인슈퍼,2,2023-06-08 13:37:00,24000.0,2,3254184.0,0,afternoon
2,2.082693e+09,갤러리카페버그내길135,2,2023-06-08 14:39:00,12000.0,1,3265126.0,0,afternoon
3,8.825201e+09,호도수꽃산장,2,2023-05-20 18:04:00,14000.0,1,3346009.0,1,night
4,3.178134e+09,농업회사법인괴산한우타운,2,2023-06-17 14:53:00,22000.0,1,3246004.0,1,afternoon
...,...,...,...,...,...,...,...,...,...
17711,5.111096e+09,반하담,2,2023-09-29 17:51:00,10000.0,2,4853861.0,0,afternoon
17712,2.476700e+09,모란창평국밥,1,2023-09-28 11:38:00,9000.0,1,4658020.0,0,morning
17713,4.098215e+09,사단법인 아시아차문화연구회,1,2023-09-28 12:40:00,4500.0,1,4658020.0,0,afternoon
17714,4.098307e+09,한국대나무박물관,1,2023-09-28 10:22:00,2000.0,3,3286016.0,0,morning


### 사업자등록번호(BRNO) 기준 방문자 수 집계

In [ ]:
# 사업자등록번호(BRNO) 기준으로 방문자 수(PAYMENT_NUM) 집계
grouped_1 = (
    df.groupby(["BRNO"])
      .agg({"PAYMENT_NUM": "sum"})
      .reset_index()
      .rename(columns={"PAYMENT_NUM": "VISITOR_COUNT"})
)

grouped_1

,BRNO,VISITOR_COUNT
0,4.133341e+08,2
1,6.223337e+08,2
2,1.011597e+09,4
3,1.011606e+09,3
4,1.011633e+09,2
...,...,...
7336,8.991701e+09,1
7337,8.992166e+09,2
7338,8.992601e+09,4
7339,8.998103e+09,1


In [ ]:
# 방문자 수 분포 확인
min_visitor = grouped_1["VISITOR_COUNT"].min()
print(f"최소 방문자 수: {min_visitor}")

max_visitor = grouped_1["VISITOR_COUNT"].max()
print(f"최대 방문자 수: {max_visitor}")

# 각 방문자 수 별 빈도수 집계
visitor_counts = grouped_1["VISITOR_COUNT"].value_counts().sort_index()
print("\n방문자 수 별 데이터 개수:")
print(visitor_counts)

최소 방문자 수: 1
최대 방문자 수: 1080

방문자 수 별 데이터 개수:
VISITOR_COUNT
1        915
2       2646
3        888
4       1132
5        348
        ... 
173        1
194        1
373        1
440        1
1080       1
Name: count, Length: 89, dtype: int64


### 사업자등록번호(BRNO) + 주말여부(IS_WEEKEND) + 시간대(TIME_SLOT) 기준 방문자 수 집계


In [5]:
# 사업자등록번호(BRNO) + 주말여부(IS_WEEKEND)  + 시간대(TIME_SLOT) 기준으로 방문자 수(PAYMENT_NUM) 집계
grouped_2 = (
    df.groupby(["BRNO", "IS_WEEKEND", "TIME_SLOT"])
      .agg({"PAYMENT_NUM": "sum"})
      .reset_index()
      .rename(columns={"PAYMENT_NUM": "VISITOR_COUNT"})
)

grouped_2

,BRNO,IS_WEEKEND,TIME_SLOT,VISITOR_COUNT
0,4.133341e+08,0,afternoon,2
1,6.223337e+08,1,afternoon,2
2,1.011597e+09,1,afternoon,4
3,1.011606e+09,1,night,3
4,1.011633e+09,1,afternoon,2
...,...,...,...,...
9903,8.992166e+09,1,afternoon,2
9904,8.992601e+09,0,night,4
9905,8.998103e+09,0,afternoon,1
9906,8.998501e+09,0,afternoon,2


In [6]:
# 방문자 수 분포 확인
min_visitor = grouped_2["VISITOR_COUNT"].min()
print(f"최소 방문자 수: {min_visitor}")

max_visitor = grouped_2["VISITOR_COUNT"].max()
print(f"최대 방문자 수: {max_visitor}")

# 각 방문자 수 별 빈도수 집계
visitor_counts = grouped_2["VISITOR_COUNT"].value_counts().sort_index()
print("\n방문자 수 별 데이터 개수:")
print(visitor_counts.head(30))
print(visitor_counts.tail(30))

최소 방문자 수: 1
최대 방문자 수: 227

방문자 수 별 데이터 개수:
VISITOR_COUNT
1     1336
2     3996
3     1304
4     1502
5      416
6      367
7      157
8      218
9       88
10      82
11      48
12      69
13      25
14      41
15      29
16      23
17      27
18      36
19      11
20      13
21       7
22      12
23       9
24       8
25       3
26       4
27       8
28       5
29       6
30       5
Name: count, dtype: int64
VISITOR_COUNT
37     1
38     1
40     5
42     2
43     4
46     2
47     1
49     1
50     2
51     1
52     1
54     1
60     1
62     2
65     1
66     1
67     1
68     1
69     1
72     1
82     1
104    2
105    1
107    1
115    1
144    1
146    1
177    1
199    1
227    1
Name: count, dtype: int64


In [7]:
# 혼잡도 라벨링
def label_congestion(count):
    if count >= 70:
        return "VERY_HIGH"
    elif count >= 30:
        return "HIGH"
    elif count >= 10:
        return "MEDIUM"
    else:
        return "LOW"

grouped_2["CONGESTION_LEVEL"] = grouped_2["VISITOR_COUNT"].apply(label_congestion)

label_counts = grouped_2["CONGESTION_LEVEL"].value_counts().sort_index()
print(label_counts)

CONGESTION_LEVEL
HIGH           46
LOW          9384
MEDIUM        466
VERY_HIGH      12
Name: count, dtype: int64


In [8]:
# 결제금액 집계
grouped_3 = (
    df.groupby(['BRNO', 'IS_WEEKEND', 'TIME_SLOT'])
    .agg(
        total_payment_amt=('PAYMENT_AMT_WON', 'sum'),
        avg_payment_amt=('PAYMENT_AMT_WON', 'mean'),
    )
    .reset_index()
    .rename(columns={"total_payment_amt": "TOTAL_PAYMENT_AMT", 'avg_payment_amt':"AVG_PAYMENT_AMT"})
)

In [9]:
final_df = pd.merge(grouped_2, grouped_3, on=['BRNO', 'IS_WEEKEND', 'TIME_SLOT'], how='inner')

# 상호명, 도로명 코드도 추가
final_df = pd.merge(
    final_df,
    df[['BRNO', 'STORE_NM', 'ROAD_NM_CD']].drop_duplicates('BRNO'),
    on='BRNO',
    how='left'
)

# BRNO별 최빈 ACTIVITY_TYPE_CD 계산
brno_activity_mode = (
    df.groupby('BRNO')['ACTIVITY_TYPE_CD']
    .agg(lambda x: x.mode().iloc[0])  # 최빈 코드가 여러 개면 첫 번째 값 사용
    .reset_index()
)

# final_df에 병합
final_df = pd.merge(final_df, brno_activity_mode, on='BRNO', how='left')

final_df

,BRNO,IS_WEEKEND,TIME_SLOT,VISITOR_COUNT,CONGESTION_LEVEL,TOTAL_PAYMENT_AMT,AVG_PAYMENT_AMT,STORE_NM,ROAD_NM_CD,ACTIVITY_TYPE_CD
0,4.133341e+08,0,afternoon,2,LOW,6000.0,6000.0,예터칼국수,3263048.0,1
1,6.223337e+08,1,afternoon,2,LOW,12000.0,12000.0,현완단겸상추튀김,3159011.0,1
2,1.011597e+09,1,afternoon,4,LOW,44000.0,44000.0,애니멀빌리지,4508397.0,3
3,1.011606e+09,1,night,3,LOW,36000.0,36000.0,아이것이갈비탕이다,4598391.0,1
4,1.011633e+09,1,afternoon,2,LOW,26000.0,26000.0,대아우렁이식당,3000117.0,1
...,...,...,...,...,...,...,...,...,...,...
9903,8.992166e+09,1,afternoon,2,LOW,69800.0,69800.0,세종파이낸스,3258074.0,2
9904,8.992601e+09,0,night,4,LOW,80000.0,40000.0,옛촌 삼천점,3266005.0,1
9905,8.998103e+09,0,afternoon,1,LOW,15000.0,15000.0,나루터곰탕,3240032.0,1
9906,8.998501e+09,0,afternoon,2,LOW,64860.0,64860.0,(주)다담리테일,3162033.0,2


In [10]:
# 결과 저장
final_df.to_csv("/content/drive/MyDrive/activity_consume_final_data.csv", index=False, encoding="utf-8-sig")
print("csv 저장 완료")

csv 저장 완료
